In [4]:
#%save Reunion.py 2

In [2]:
import sys, os, json
path = "../Calcul"
if path not in sys.path : 
    sys.path.append (path)
    
path = "../outils"
if path not in sys.path : 
    sys.path.append (path)

from validationPath import validationPath
from Calcul import Calcul
from Generic_simulation import Generic
import luigi

from pprint import PrettyPrinter 
def P (stuff , pp = PrettyPrinter(indent=4)) :
    return pp.pprint(stuff)

from Creation import Creation_paragraphe
from calcul_pas import calcul_pas
from Embedding import Embedding_vecteur
    
    
class Reunion_vecteur  (Generic) :
    """
    recoit dans arg 
    les paths (systeme et evenements)
    
    les parametres d'encadrement de la lecture dans l'index data
    
    bulk des vecteurs dans un index commune (pas init de l'index ceci sera réalisé par le niveau au dessus)
    
    warning le calcul d'apprentissage est supposée réalisé...
    car sinon on baisse la qualité du calcul et le temps de calcul deviend équivallent à l apprentissage
    
    """
    arg_travail = luigi.Parameter(default = "")
    
    
    def get_execution (self) :
        self.appels = {'appel' : Creation_paragraphe,
                      'type_repetition' : 'date_evenement',
                       }
        return 
        
    def output(self):
        # calcul le nom de l'output
        # et on realise le path si neccessaire
        self.get_travail()
        self.nom_tache_luigi = 'Reunion_vecteur'
        etape = self.arg_travail_data ['etape'] # nom different de l 'apprentissage'
        self.path = '../data/temp/' + etape + '/' + self.nom_tache_luigi + '/'
        validationPath (self.path)
        l = [
             self.arg_travail_data ['ID_min'],
             self.arg_travail_data ['ID_max'],
             
            ]
             
        nom_file = self.path +  ('_'.join(l)) +'.json'
        
        return luigi.LocalTarget(nom_file)
    
    
        
        
        
    def run_tache (self, ) :
        self.get_travail()
        
        dico_out = {}
        
        for i in range (0, self.nombre_appels) :
            
            with self.input() [i].open() as fin :
                data = fin.read()
                dico = json.loads (data)
                
                dico_out = reunion_dico (dico, r_entree = dico_out)
            continue
            
        # load model et bulk des vecteurs (entree dico_out avec les IDs) dans embedding_vecteur
        
        index_bulk = self.arg_travail_data ['index_bulk']
        embedding_vecteur = Embedding_vecteur (index_bulk, dico_out, self.arg_travail_data)
        embedding_vecteur.bulk_vecteur ()
                       
        data = "creation vecteur dans index =" + index_bulk
               
        with self.output().open ('w') as fout :
            fout.write (data)
        
        return message
    
                
        
def reunion_dico (r, r_entree = {}) :
    for cle, liste in r.items () :
        try :
            liste_entree = r_entree [cle]
        except:
            liste_entree = []
        liste_entree.extend(liste)
        r_entree [cle] = liste_entree
        continue
    
    return r_entree
    
    
    
    
       
        
        
    


In [5]:
import json

from pprint import PrettyPrinter 
def P (stuff , pp = PrettyPrinter(indent=4)) :
    return pp.pprint(stuff)

from Reunion import Reunion_paragraphe
import luigi

arg = {}


#  variable pour alimentation bloc

arg ['ID_reference_min'] = 0
arg ['ID_reference_max'] = 100000000
arg ['ID_reference_sort'] = 'asc'
arg ['isReference'] = False




arg ['isVariable'] =  True
arg ['nom_variableQuery'] = "date_evenement"
arg ['variable_min'] = '2021-02-01 00:00:00'
arg ['variable_max'] = '2021-02-09 00:00:00'
arg ['variable_sort'] = 'asc'

arg ['isID'] = True
arg ['ID_min'] = "couple_cadre_0"
arg ['ID_max']  = "homme_ouvrier_1"
arg ['ID_sort']  = None
arg ['isTrace'] = False

arg ['pathListe_ID'] =  "../data/listeID.txt"
arg ['pathDico_evenements'] = '../data/dico_evenements_2.txt'
arg ['pathDico_systeme'] = '../data/dico_systeme_2.txt'
arg ['etape'] =  'test'

arg_json = json.dumps (arg)

if __name__ == '__main__':
    luigi.build([Reunion_paragraphe (arg_travail = arg_json)],
                 workers=2, local_scheduler = True)
    
    

print ('fin test Reunion_paragraphe')

fin test Reunion_paragraphe
